In [1]:
!pip install --pre deepchem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 12.9 MB/s eta 0:00:00


In [2]:
import deepchem as dc
dc.__version__

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


'2.8.1.dev'

In [3]:
#fingerprint = fixed length array representing a molecule
#ECFP = a particular type of fingerprint (featurizer)

In [6]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer="ECFP")
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


In [7]:
train_dataset.w
#some of the elements have weights of 0s because assays (evaluation of presence, amount, or functional activity) were not performed

array([[1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 1.17464996,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 1.17464996,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 0.        ,
        1.05288369],
       ...,
       [1.04502242, 0.        , 1.12502653, ..., 1.05576503, 6.7257384 ,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 6.7257384 ,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 0.        , 1.17464996,
        1.05288369]])

In [20]:
print("Total number of samples: {}".format(len(train_dataset.X))) #total of 6264 samples
print("Shape of a single sample: {}".format(len(train_dataset.X[0])))

Total number of samples: 6264
Shape of a single sample: 1024


In [14]:
len(train_dataset.y[0]) #12 possible labels (this is a multitask classification dataset)

12

In [15]:
#the model architecture was previously defined
#12 tasks = 12 outputs
#n_features = 1024 input neurons
#layer_sizes = 1 hidden layer of 1000 neurons

#architecture:
#1024 -> 1000 -> 12
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])

In [16]:
import numpy as np

#fitting the model
#nb_epoch = number of epochs
model.fit(train_dataset, nb_epoch=10) #training
metric = dc.metrics.Metric(dc.metrics.roc_auc_score) #choosing the metric for evaluation

print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(train_dataset, [metric], transformers))

training set score: {'roc_auc_score': 0.9590033758663236}
test set score: {'roc_auc_score': 0.9590033758663236}
